In [51]:
#Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
import time

In [52]:
# Text Sample
text = "This is a simple example to demonstrate how to predict the next character using RNN in PyTorch."

In [53]:
# **Taken from the course GitHub**

#set(text): Creates a set of unique characters found in the text. The set function removes any duplicate characters.
#list(set(text)): Converts the set back into a list so that it can be sorted. 
# sorted(list(set(text))): Sorts the list of unique characters. 
chars = sorted(list(set(text)))
#This line creates a dictionary that maps each character to a unique index (integer)."
ix_to_char = {i: ch for i, ch in enumerate(chars)}
#Similar to the previous line, but in reverse. This line creates a dictionary that maps each unique index (integer) back to its corresponding character.
char_to_ix = {ch: i for i, ch in enumerate(chars)} 
chars = sorted(list(set(text)))

In [54]:
# Preparing the dataset
max_length = 10  # Maximum length of input sequences
X = []
y = []
for i in range(len(text) - max_length):
    sequence = text[i:i + max_length]
    label = text[i + max_length]
    X.append([char_to_ix[char] for char in sequence])
    y.append(char_to_ix[label])

X = np.array(X)
y = np.array(y)

In [55]:
# Splitting the dataset 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Converting to tensors
X_train = torch.tensor(X_train, dtype=torch.long)#.to(device)
y_train = torch.tensor(y_train, dtype=torch.long)#.to(device)
X_val = torch.tensor(X_val, dtype=torch.long)#.to(device)
y_val = torch.tensor(y_val, dtype=torch.long)#.to(device)

In [56]:
#Training Loop + Empty Arrays for storing Training and Validation Results

def TL(n_epochs, optimizer, model, loss_fn, X_train, y_train, X_val, y_val):
  for epoch in range(1, n_epochs + 1):
    #Training Loop
    model.train()
    optimizer.zero_grad()
    train_out = model(X_train)
    loss = loss_fn(train_out, y_train)
    loss.backward()
    optimizer.step()
    
    #Validation Loop
    model.eval()
    with torch.no_grad():
      val_out = model(X_val)
      val_loss = loss_fn(val_out, y_val)
      _, predicted = torch.max(val_out, 1)
      val_accuracy = (predicted == y_val).float().mean()
    

    if epoch == 1 or epoch % 5 == 0: 
      print('Epoch: {}, Training Loss: {}, Validation Loss: {}, Validation Accuracy: {}'.format(epoch, loss.item(), val_loss.item(), val_accuracy.item()))


Question 1 - Part B: LSTM

In [57]:
# LSTM Model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size 
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.lstm(embedded)
        output = self.fc(output[:, -1, :])
        return output

In [58]:

hidden_size = 128
model = LSTM(len(chars), hidden_size, len(chars))
a = 0.01
epochs = 20
lossFN = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr= a)
start_time = time.time()

TL(
  n_epochs= epochs,
  optimizer= optimizer,
  model= model,
  loss_fn= lossFN,
  X_train= X_train,
  y_train= y_train,
  X_val= X_val,
  y_val= y_val
)

#Getting the time
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time} Seconds")

Epoch: 1, Training Loss: 3.2315425872802734, Validation Loss: 3.181814432144165, Validation Accuracy: 0.11764705926179886
Epoch: 5, Training Loss: 3.225187301635742, Validation Loss: 3.1817421913146973, Validation Accuracy: 0.05882352963089943
Epoch: 10, Training Loss: 3.2172932624816895, Validation Loss: 3.181669235229492, Validation Accuracy: 0.05882352963089943
Epoch: 15, Training Loss: 3.209451198577881, Validation Loss: 3.181615114212036, Validation Accuracy: 0.0
Epoch: 20, Training Loss: 3.201658010482788, Validation Loss: 3.181579351425171, Validation Accuracy: 0.05882352963089943
Time taken: 0.1620006561279297 Seconds


In [59]:
# Prediction Fn
def predict_next_char(model, char_to_ix, ix_to_char, initial_str):
    model.eval()
    with torch.no_grad():
        initial_input = torch.tensor([char_to_ix[c] for c in initial_str[-max_length:]], dtype=torch.long).unsqueeze(0)
        prediction = model(initial_input)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return ix_to_char[predicted_index]

In [60]:
#Complexity
numel_list = [p.numel() for p in model.parameters()]
print("Model Complexity: {}".format(sum(numel_list)))
#sum(numel_list), numel_list

Model Complexity: 138521


In [61]:
# Predicting the next character
test_str = "This is a simple example to demonstrate how to predict the next char"
predicted_char = predict_next_char(model, char_to_ix, ix_to_char, test_str)
print(f"Predicted next character: '{predicted_char}'")

Predicted next character: 'n'
